In [3]:
from loadTDMS import *
# from fft_test import *
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
# import seaborn as sns

# 実験日選択

In [4]:
# date = 20211025
# date = 20211115
# date = 20220630
# date = 20221027
# date = 20230113
# date = 20230117
# date = 20230120
date = 20230123

# # cuttingﾌｫﾙﾀﾞの中の実切削中のサンプリング数特定

In [13]:
num = 300
path = rf"Z:\100 Raw_data\{date}---切削実験\Dynamometer\Cutting\Dynamo_{num}.tdms"
# npath = rf"Z:\100 Raw_data\{date}---切削実験\Dynamometer\Non-Cutting\Dynamo_{num}.tdms"

cd = load_tdms(path, ch_num=3)
# ncd = load_tdms(npath, ch_num=3)

print(len(cd[1]))
# print(len(ncd[1]))

10422567


In [14]:
%matplotlib qt
plt.figure()
plt.plot(cd[1])

## 日付 : 切削中のサンプル数 / last path 
- 20211025 : 5060000 / 24path
- 20211115 : 5010000 / 238path
- 20220630 : 4670000 / 735path
- 20221027 : 4930000 / 168path
- 20230113 : 9970000 / 120path
- 20230117 : 9970000 / 340path
- 20230120 : 9970000 / 460path
- 20230123 : [100000:9600000] / 460path

In [12]:
if date == 20211025:
    last_path = 24
    last_sample_num = 5060000
if date == 20211115:
    last_path = 238
    last_sample_num = 5010000
if date == 20220630:
    last_path = 735
    last_sample_num = 4670000 
if date == 20221027:
    last_path = 168
    last_sample_num = 4930000
    
if date == 20230113:
    last_path = 120
    last_sample_num = 9970000

if date == 20230117:
    last_path = 460
    last_sample_num = 9970000
    
if date == 20230120:
    last_path = 460
    last_sample_num = 9970000

if date == 20230123:
    last_path = 460
    last_sample_num = 9960000    
    
print(rf"date={date}, last_path={last_path}, last_sample_num={last_sample_num}")

date=20230123, last_path=460, last_sample_num=9960000


## get cutting voltage avg
cuttingの平均電圧値の推移調べ

In [13]:
cutting_avg = np.array([])
for num in tqdm(range(1,last_path+1)):
    path = rf"Z:\100 Raw_data\{date}---切削実験\Dynamometer\Cutting\Dynamo_{num}.tdms"
    cd = load_tdms(path, ch_num=3)
    a = np.array(cd[1][100000:last_sample_num])
    cutting_avg = np.append(cutting_avg, np.average(a))
    
np.save(rf"Z:\200 Produced_data\{date}\CF\cutting_avg",cutting_avg)
cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\cutting_avg.npy")

100%|██████████| 460/460 [12:44<00:00,  1.66s/it]


In [9]:
%matplotlib qt
cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\cutting_avg.npy")
plt.plot(cutting_avg)
# plt.savefig(rf"Z:\200 Produced_data\{date}\CF\cutting_avg.png")

## get non cutting voltage avg
non-cuttingの平均電圧値の推移調べ

In [15]:
non_cutting_avg = np.array([])
for num in tqdm(range(1,last_path+1)):
    path = rf"Z:\100 Raw_data\{date}---切削実験\Dynamometer\Non-Cutting\Dynamo_{num}.tdms"
    ncd = load_tdms(path, ch_num=3)
    a = np.array(ncd[1])
    non_cutting_avg = np.append(non_cutting_avg, np.average(a))

np.save(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg",non_cutting_avg)

100%|██████████| 460/460 [01:09<00:00,  6.63it/s]


In [16]:
%matplotlib qt
non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg.npy")
plt.plot(non_cutting_avg)
plt.savefig(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg.png")

## 発散除去工程

### 2023 0113
non-cuttingの平均電圧は、発散しているpathを除けば、おおよそ右肩下がりだったので  
1path目(最初)と120path目(最後)を結ぶ1次関数に置き換えて、new_non_cutting_avgを算出した。

In [32]:
%matplotlib qt

date = 20230113
non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg.npy")

katamuki = (non_cutting_avg[-1] - non_cutting_avg[0]) / len(non_cutting_avg)
y_seppen = non_cutting_avg[-1] - (katamuki * 120)

print(katamuki)
print(y_seppen)

new_non_cutting_avg = np.array([])
for i in range(1,121):
    new_non_cutting_avg = np.append(new_non_cutting_avg, katamuki * (i+1) + y_seppen)
np.save(rf"Z:\200 Produced_data\{date}\CF\new_non_cutting_avg", new_non_cutting_avg)

non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg.npy")
new_non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\new_non_cutting_avg.npy")

plt.plot(non_cutting_avg)
plt.plot(new_non_cutting_avg)

-0.0005201947914126019
-0.00602323142811656


### 2023 0117
non-cuttingの平均電圧は、発散しているpathを除いても、1次関数の直線には近似できなさそうだったので  
発散しているpathの平均電圧に関しては、そのpathを中心として、自分を除く前後それぞれ12pathずつ、  
合計24path分の平均値に置き換えて、new_non_cutting_avgを算出した。

In [43]:
date = 20230117
non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg.npy")

ncd = np.where(non_cutting_avg < -0.8, 0, non_cutting_avg)
ncd_origin = non_cutting_avg.copy()

index = np.where(ncd == 0)
for i in index[0]:
    ncd[i] = np.average(ncd[i-12:i+12])
# plt.plot(nnd_origin)
# plt.plot(nnd)

np.save(rf"Z:\200 Produced_data\{date}\CF\new_non_cutting_avg",ncd)

non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg.npy")
new_non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\new_non_cutting_avg.npy")

plt.plot(non_cutting_avg)
plt.plot(new_non_cutting_avg)

### 2023 0120
non-cuttingの平均電圧は、発散しているpathを除けば、おおよそ右肩下がりだったので  
1path目(最初)と460path目(最後)を結ぶ1次関数に置き換えて、new_non_cutting_avgを算出した。

In [23]:
date = 20230120
non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg.npy")

katamuki = (non_cutting_avg[-1] - non_cutting_avg[0]) / len(non_cutting_avg)
y_seppen = non_cutting_avg[-1] - (katamuki * 460)

print(katamuki)
print(y_seppen)

new_non_cutting_avg = np.array([])
for i in range(1,461):
    new_non_cutting_avg = np.append(new_non_cutting_avg, katamuki * (i+1) + y_seppen)
np.save(rf"Z:\200 Produced_data\{date}\CF\new_non_cutting_avg", new_non_cutting_avg)

non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg.npy")
new_non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\new_non_cutting_avg.npy")

plt.plot(non_cutting_avg)
plt.plot(new_non_cutting_avg)

-0.00028426609933376314
0.03528834879398346


## 2023 0123

In [15]:
# date = 20230113
# date = 20230117
# date = 20230120
date = 20230123

In [16]:
ca = np.load(rf"Z:\200 Produced_data\{date}\CF\cutting_avg.npy")
nca = np.load(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg.npy")
# nnca = np.load(rf"Z:\200 Produced_data\{date}\CF\new_non_cutting_avg.npy")

In [22]:
# plt.plot(ca)
plt.plot(nca)

In [53]:
date = 20230123
non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\non_cutting_avg.npy")

ncd = np.where(non_cutting_avg < -0.6, 0, non_cutting_avg)
ncd_origin = non_cutting_avg.copy()

index = np.where(ncd == 0)
for i in index[0]:
    ncd[i] = (np.average(ncd[i-2:i]) + np.average(ncd[i+1:i+2]))/2


ncd[42] = (ncd[41] + ncd[43])/2
ncd[282] =  (ncd[281] + ncd[284])/2
ncd[283] = (ncd[282] + ncd[284])/2
ncd[60] = (ncd[59] + ncd[61] )/2
ncd[61] = (ncd[62] + ncd[59]) /2
ncd[43] = (ncd[42] + ncd[44])/2
# print(ncd[42])
# print(ncd[282])
    
# plt.plot(ncd)
# plt.plot(nnd_origin)

np.save(rf"Z:\200 Produced_data\{date}\CF\new_non_cutting_avg",ncd)



0.14459306886419654
0.27179546654224396


In [56]:
ca = np.load(rf"Z:\200 Produced_data\{date}\CF\cutting_avg.npy")
new_non_cutting_avg = np.load(rf"Z:\200 Produced_data\{date}\CF\new_non_cutting_avg.npy")

plt.plot(ca)
plt.plot(new_non_cutting_avg)

In [57]:
ca_p = np.where(ca < -0.24, 0, ca)
ca_o = np.copy(ca)
index = np.where(ca_p == 0)
index = index[0] + 1
index

array([ 19,  60,  63,  65,  66,  72,  73,  74,  75,  81,  84,  85,  90,
        91,  92,  95,  97, 100, 102, 103, 104, 105, 106, 117, 118, 119,
       120, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
       135, 137, 138, 139, 141, 142, 143, 144, 145, 146, 147, 150, 154,
       155, 158, 161, 164, 168, 169, 170, 173, 174, 176, 179, 180, 181,
       182, 183, 184, 185, 186, 188, 190, 191, 192, 193, 194, 195, 198,
       201, 211, 217, 218, 220, 221, 234, 237, 238, 239, 241, 242, 245,
       246, 248, 251, 252, 255, 258, 260, 261, 263, 264, 265, 266, 268,
       270, 275, 277, 278, 279, 280, 281, 282, 283, 286, 287, 288, 289,
       290, 291, 293, 294, 295, 296, 299, 300, 302, 303, 304, 305, 306,
       307, 308, 309, 311, 314, 315, 318, 320, 321, 322, 323, 324, 325,
       328, 329, 331, 332, 333, 335, 336, 338, 343, 344, 345, 346, 347,
       350, 351, 352, 354, 355, 357, 358, 359, 360, 362, 363, 364, 365,
       367, 368, 369, 370, 371, 372, 373, 374, 377, 378, 380, 38

In [117]:
%matplotlib qt
ca_p = np.where(ca < -0.24, 0, ca)
ca_o = np.copy(ca)
index = np.where(ca_p == 0)
index = index[0] + 1
# np.save(rf"Z:\200 Produced_data\{date}\CF\drift_path",index)
plt.plot(ca_p)
# plt.plot(nca)
nnnca = nnca+np.random.randint(-6,6,460)*0.001

# plt.plot(nnnca)
# plt.plot(ca_p + nnnca)
# plt.plot(ca_p + nnca)
plt.plot(ca_p)
# plt.plot((ca_p - nca)*500)

In [126]:
np.save(rf"Z:\200 Produced_data\{date}\CF\new_non_cutting_avg2", nnnca)

In [119]:
len(index)

286

In [25]:
# plt.plot((ca - nca)*100)
plt.plot(ca)

In [120]:
drift_path = np.load(rf"Z:\200 Produced_data\{date}\CF\drift_path.npy")
drift_path

array([  2,   3,   4,   5,   6,   7,   9,  10,  11,  12,  13,  14,  16,
        17,  18,  19,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  40,  41,  43,  44,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  59,  60,
        61,  62,  64,  65,  66,  67,  69,  70,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  84,  85,  86,  87,  88,  90,  91,
        92,  93,  94,  95,  97,  98,  99, 100, 101, 102, 103, 104, 106,
       109, 111, 113, 115, 116, 117, 119, 120, 121, 122, 124, 125, 127,
       129, 130, 131, 132, 133, 134, 135, 139, 144, 146, 148, 150, 152,
       155, 156, 158, 159, 160, 161, 163, 164, 165, 167, 169, 172, 173,
       174, 175, 177, 179, 182, 185, 187, 189, 191, 193, 194, 195, 196,
       198, 201, 202, 205, 206, 207, 208, 209, 210, 212, 216, 218, 219,
       220, 221, 222, 223, 226, 227, 229, 231, 232, 234, 237, 238, 241,
       243, 244, 245, 246, 247, 248, 250, 251, 252, 253, 255, 25

In [131]:
a = np.arange(1,461)
normal = np.delete(a, drift_path-1)
normal

array([  1,   8,  15,  20,  39,  42,  45,  58,  63,  68,  71,  83,  89,
        96, 105, 107, 108, 110, 112, 114, 118, 123, 126, 128, 136, 137,
       138, 140, 141, 142, 143, 145, 147, 149, 151, 153, 154, 157, 162,
       166, 168, 170, 171, 176, 178, 180, 181, 183, 184, 186, 188, 190,
       192, 197, 199, 200, 203, 204, 211, 213, 214, 215, 217, 224, 225,
       228, 230, 233, 235, 236, 239, 240, 242, 249, 254, 256, 260, 262,
       267, 268, 273, 276, 281, 283, 284, 287, 288, 290, 294, 295, 296,
       297, 299, 301, 303, 304, 307, 308, 310, 311, 313, 314, 315, 317,
       319, 321, 323, 330, 331, 335, 344, 350, 353, 354, 356, 358, 364,
       369, 373, 375, 377, 378, 382, 383, 388, 390, 391, 392, 394, 396,
       397, 398, 399, 403, 404, 405, 408, 409, 410, 411, 412, 414, 416,
       419, 420, 421, 422, 424, 426, 427, 428, 429, 430, 431, 433, 434,
       436, 438, 439, 440, 441, 443, 445, 449, 450, 451, 452, 453, 454,
       455, 456, 457, 459, 460])

In [132]:
num = 344
path = rf"Z:\100 Raw_data\{date}---切削実験\Dynamometer\Cutting\Dynamo_{num}.tdms"
cd = load_tdms(path, ch_num=3)
# cd = cd[1][36000:10006000]
cd = cd[1]
plt.plot(cd*-500)

In [130]:
num = 343
path = rf"Z:\100 Raw_data\{date}---切削実験\Dynamometer\Cutting\Dynamo_{num}.tdms"
cd = load_tdms(path, ch_num=3)
# cd = cd[1][36000:10006000]
cd = cd[1]
print(nca[num+1])
cd_v = (cd - np.average(cd[-8303*10:-1]))
cd_a = (cd - np.average(cd[-8303*10:-1]) - nca[num+1])

# plt.plot(cd)
# plt.plot(cd_v)
plt.plot(cd_a*-500)

-0.054437585175037384


In [30]:
for num in tqdm(drift_path):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    path = rf"Z:\100 Raw_data\{date}---切削実験\Dynamometer\Cutting\Dynamo_{num}.tdms"
    cd = load_tdms(path, ch_num=3)
    cd = cd[1][-8303*10:-1]
    plt.plot(cd)
    plt.vlines(x = 0, ymin=np.min(cd) , ymax=np.max(cd) , colors="red")
    plt.savefig(rf"0120\{num}.jpg")
    plt.close()
# noise_a = np.average(cd[1][35000:55000])
# print(noise_a)
# plt.plot(cd[1]- noise_a)
# plt.plot(cd[1]- (-3.92))
# plt.plot(cd[1][-10000:-1])

100%|██████████| 251/251 [09:07<00:00,  2.18s/it]


20230120  
cd[1] - np.average(cd[1][-10000:-1])

20230123  

- cd[1][33000:35000]  
[63, 65, 66, 75, 84, 90, 91, 92, 95, 97, 100, 102, 103, 104, 105, 106, 117, 118, 119,120, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 137, 138, 139, 141, 142, 143, 144, 145, 146, 147, 150, 154, 155, 158, 161, 164, 168, 169, 170, 173, 176, 179, 180, 181,182, 183, 185, 188, 190, 191, 192, 193, 194, 195, 198, 201, 211, 217, 218, 220, 221, 234, 237, 241, 246, 251, 252, 255, 258, 260, 261, 263, 264, 266, 282, 283, 288, 289,290, 291, 293, 300, 304, 305, 307, 308, 309, 311, 314, 315, 318, 320, 323, 325, 328, 329, 333, 343, 345, 351, 352, 358, 364, 369, 370, 372, 381, 382, 387, 402, 404, 414]

- cd[1][-2000:-1]  
[19, 336, 360, 374, 389, 398, 399, 409, 416, 425, 429, 433, 443, 448, 449, 454, 457, 60, 72, 73,
81, 85, 123, 135, 174, 184, 238, 242, 245, 248, 265,268, 270, 277, 281, 286, 287, 296, 299, 302, 306, 321, 322,324, 331, 332, 335, 338, 346, 347, 350, 355, 359, 363, 365,367, 371, 373, 377, 384, 385, 388, 401, 403, 405, 407, 408, 412, 435, 451, 452, 458, 459, 460]

- cd[1][-5000:-3000]  
[186, 239, 275, 278, 279, 280, 294, 295, 303, 344, 354, 357, 362, 368, 378, 380, 390, 411, 442, 445]

- x、ｙ逆転
74, 



In [66]:
b= [19, 60, 72, 73, 74, 81, 85, 123, 135, 174, 184, 186, 238, 239, 242, 245, 248, 265,268, 270, 275, 277, 278, 279, 280, 281, 286, 287, 294, 295, 296, 299, 302, 303, 306, 321, 322,324, 331, 332, 335, 336, 338, 344, 346, 347, 350, 354, 355, 357, 359, 360, 362, 363, 365,367, 368, 371, 373, 374, 377, 378, 380, 384, 385, 388, 389, 390, 398, 399, 401, 403, 405, 407, 408, 409, 411, 412, 416, 425, 429, 433, 435, 442, 443, 445, 448, 449, 451, 452, 454, 457, 458, 459, 460]

In [70]:
for i in tqdm(b):
    path = rf"Z:\100 Raw_data\{date}---切削実験\Dynamometer\Cutting\Dynamo_{i}.tdms"
    cd = load_tdms(path, ch_num=3)
    cd = cd[1][-8303*5 -10000:-1]
    plt.vlines(x = 8303*5+8000 , ymin=np.min(cd) , ymax=np.max(cd) , colors="red")
    plt.plot(cd)
    plt.savefig(rf"b\{i}.jpg")
    plt.close()

100%|██████████| 95/95 [02:38<00:00,  1.67s/it]


In [78]:
path = rf"Z:\100 Raw_data\{date}---切削実験\Dynamometer\Cutting\Dynamo_1.tdms"
cd = load_tdms(path, ch_num=3)
plt.plot(cd[0])

In [73]:
a = np.array([63, 65, 66, 75, 84, 90, 91, 92, 95, 97, 100, 102, 103, 104, 105, 106, 117, 118, 119,120, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 137, 138, 139, 141, 142, 143, 144, 145, 146, 147, 150, 154, 155, 158, 161, 164, 168, 169, 170, 173, 176, 179, 180, 181,182, 183, 185, 188, 190, 191, 192, 193, 194, 195, 198, 201, 211, 217, 218, 220, 221, 234, 237, 241, 246, 251, 252, 255, 258, 260, 261, 263, 264, 266, 282, 283, 288, 289,290, 291, 293, 300, 304, 305, 307, 308, 309, 311, 314, 315, 318, 320, 323, 325, 328, 329, 333, 343, 345, 351, 352, 358, 364, 369, 370, 372, 381, 382, 387, 402, 404, 414])
b = np.array([19, 336, 360, 374, 389, 398, 399, 409, 416, 425, 429, 433, 443, 448, 449, 454, 457, 60, 72, 73, 81, 85, 123, 135, 174, 184, 238, 242, 245, 248, 265,268, 270, 277, 281, 286, 287, 296, 299, 302, 306, 321, 322,324, 331, 332, 335, 338, 346, 347, 350, 355, 359, 363, 365,367, 371, 373, 377, 384, 385, 388, 401, 403, 405, 407, 408, 412, 435, 451, 452, 458, 459, 460])
c = np.array([186, 239, 275, 278, 279, 280, 294, 295, 303, 344, 354, 357, 362, 368, 378, 380, 390, 411, 442, 445])

In [75]:
a = np.sort(a)
b = np.sort(b)
c = np.sort(c)

In [76]:
np.save(rf"Z:\200 Produced_data\20230123\CF\drift_path1",a)
np.save(rf"Z:\200 Produced_data\20230123\CF\drift_path2",b)
np.save(rf"Z:\200 Produced_data\20230123\CF\drift_path3",c)